## 线性回归的Gluon实现
### 1.生成数据集

In [2]:
from mxnet import autograd,nd

num_inputs=2
num_examples=1000
true_w=[2,-3.4]
true_b=4.2
features=nd.random.normal(scale=1,shape=(num_examples,num_inputs))
labels=true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels+=nd.random.normal(scale=0.01,shape=labels.shape)

### 2.读取数据

In [3]:
from mxnet.gluon import data as gdata

batch_size=10
# 将训练数据的特征和标签组合
dataset=gdata.ArrayDataset(features,labels)
# 随机读取小批量
data_iter=gdata.DataLoader(dataset,batch_size,shuffle=True) #shuffle是把数组中的元素按随机顺序重新排列

In [4]:
for X,y in data_iter:
    print(X,y)
    break


[[ 0.69374377 -0.73523283]
 [-0.00286585  0.87209868]
 [-1.56856549  1.04608738]
 [ 0.5331459  -0.72925323]
 [-0.93251252  2.05497503]
 [-0.51157194  0.39294311]
 [-0.90226823 -0.785276  ]
 [ 1.5116657   1.84036314]
 [-2.33943224  1.89679122]
 [-1.16881478  1.55807114]]
<NDArray 10x2 @cpu(0)> 
[ 8.09055328  1.22845292 -2.49339509  7.74128771 -4.65387917  1.83855939
  5.07593489  0.9611423  -6.93585825 -3.43311238]
<NDArray 10 @cpu(0)>


### 3.定义模型
    先定义一个模型变量net，它是一个 Sequential 实例。
    在 Gluon 中，Sequential 实例可以看作是一个串联各个层的容器。
    在构造模型时，我们在该容器中依次添加层。当给定输入数据时，容器中的每一层将依次计算并将输出作为下一层的输入。
    
    在 Gluon 中我们无需指定每一层输入的形状,模型将自动推断出每一层的输入个数

In [5]:
from mxnet.gluon import nn

net=nn.Sequential()

In [6]:
# 在 Gluon 中，全连接层是一个Dense实例。我们定义该层输出个数为 1。
net.add(nn.Dense(1))

### 4.初始化模型参数
    initializer模块提供了模型参数初始化的各种方法。
    init是initializer的缩写形式。
    通过init.Normal(sigma=0.01)指定权重参数每个元素将在初始化时随机采样于均值为 0 ，标准差为 0.01 的正态分布。
    偏差参数默认会初始化为零。

from mxnet import init

net.initialize(init.Normal(sigma=0.01))

### 5.定义损失函数
    loss模块定义了各种损失函数
    直接使用提供的平方损失函数作为模型的损失函数

In [9]:
from mxnet.gluon import loss as gloss

loss=gloss.L2Loss() # 平方损失又称L2范数损失

###  6.定义优化算法
    在导入 Gluon 后，我们创建一个Trainer实例，并指定学习率为 0.03 的小批量随机梯度下降（sgd）为优化算法。
    该优化算法将用来迭代net实例所有通过add函数嵌套的层所包含的全部参数，这些参数可以通过collect_params函数获取。

In [15]:
from mxnet import gluon

trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})

### 7.训练模型
    在使用 Gluon 训练模型时，我们通过调用Trainer实例的step函数来迭代模型参数。
    上一节中我们提到，由于变量l是长度为batch_size的一维 NDArray，执行l.backward()等价于执行l.sum().backward()。
    按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均。

In [16]:
num_epochs=3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l=loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l=loss(net(features),labels)
    print('epoch %d, loss: %f' % (epoch,l.mean().asnumpy()))

epoch 1, loss: 0.035153
epoch 2, loss: 0.000121
epoch 3, loss: 0.000048


In [18]:
net

Sequential(
  (0): Dense(2 -> 1, linear)
)

In [23]:
dense=net[0]
dense

Dense(2 -> 1, linear)

In [24]:
true_w,dense.weight.data()

([2, -3.4], 
 [[ 1.99936318 -3.39988542]]
 <NDArray 1x2 @cpu(0)>)

In [25]:
true_b,dense.bias.data()

(4.2, 
 [ 4.19997835]
 <NDArray 1 @cpu(0)>)